# Import

In [39]:
import pandas as pd
# from fugue_sql import fsql
#Single model classifiers
from sklearn.linear_model import LogisticRegression
from sklearn.tree import DecisionTreeClassifier
#Ensemble classifications models
from sklearn.ensemble import RandomForestClassifier, GradientBoostingClassifier
#Model evaluation
from sklearn.model_selection import train_test_split
from sklearn.metrics import (accuracy_score,confusion_matrix, classification_report, roc_auc_score,
f1_score, recall_score, precision_score, SCORERS)
import pickle
from sklearn.preprocessing import OneHotEncoder
from sklearn import preprocessing
from sklearn.preprocessing import LabelEncoder
import numpy as np
from collections import Counter
from sklearn.ensemble import AdaBoostClassifier
import matplotlib.pyplot as plt

In [40]:
Modelname = 'RandomForestClassifier_model.sav'


# Functions

In [41]:
def baseline_models(data=[], verbose=False):
    #List of models to be used
    models=[DecisionTreeClassifier(),LogisticRegression(),
            RandomForestClassifier(n_estimators=100,criterion='entropy',random_state=101),
            GradientBoostingClassifier(learning_rate=0.1,random_state=1,n_estimators=100,min_samples_split=5,min_samples_leaf=1,subsample=0.8,max_depth=3),
            AdaBoostClassifier(DecisionTreeClassifier(max_depth=1),n_estimators=200,random_state=1,learning_rate=1)
]
    #Create training and testing data sets depending on wheather or not they have been generated previously.
    if len(data)== 2:
        X_train, X_test, y_train, y_test = train_test_split(data[0],data[1],random_state=123)
    else:
        X_train, X_test, y_train, y_test = data[0],data[1],data[2],data[3]
    #Instantiate lists to store each of the models results
    accuracy = []
    f1 = []
    auc = []
    recall = []
    precision = []
    #Run thorugh each of the models to get their performance metrics
    for model in models:
        clf = model
        clf.fit(X_train, y_train)
        test_preds = clf.predict(X_test)
        f1.append(f1_score(y_test, test_preds,average='macro'))
        accuracy.append(accuracy_score(y_test, test_preds))
        # auc.append(roc_auc_score(y_test, test_preds,multi_class="ovr" ))
        recall.append(recall_score(y_test, test_preds,average='macro'))
        precision.append(precision_score(y_test, test_preds,average='macro'))
        #Print the model and its report
        if verbose:
            print('Classification Model: ',model,'\n')
            print(classification_report(y_test, test_preds),'\n')
    #store results in dataframe
    results = pd.DataFrame([f1, accuracy, precision,recall],
                      index= ['f1','accuracy','precision','recall',],
                           columns=['DecisionTree','LogisticRegression','RandomForest','Gradient Boosting','AdaBoostClassifier'])
    #Change orientation of the dataframe
    return results.transpose()

# ML

In [42]:
path='/home/o/oananbeh/notebook/Experment2/src/TabTransformer-multiclass_died/'

In [43]:
# #connect the SOMTE_DataSet_died with SOMTE_DataSet_discharge
# died_DF=pd.read_csv(path+'SOMTE_DataSet_prepare_association_died_0.5_0.7.csv')
# died_DF['OUTCOME']=0
# discharge_DF=pd.read_csv(path+'SOMTE_DataSet_prepare_association_discharge_0.5_0.7.csv')
# discharge_DF['OUTCOME']=1
# frames = [died_DF, discharge_DF]
# Filal_SMOTE_DataSet=pd.concat(frames)

In [44]:
# Filal_SMOTE_DataSet.shape

In [45]:
# Filal_SMOTE_DataSet.to_csv(path+'Filal_SMOTE_DataSet.csv',index=False)

In [46]:
Filal_SMOTE_DataSet=pd.read_csv(path+'SMOT_died_DataSet.csv')

In [47]:
target = Filal_SMOTE_DataSet.OUTCOME
print(f"Original class counts: {Counter(target)}")

Original class counts: Counter({'died=<3': 38, 'died=>3': 38})


In [48]:
ohc_feats_nonNumericalFeatures = pd.get_dummies(Filal_SMOTE_DataSet)


In [49]:
# ohc_feats_nonNumericalFeatures.to_csv(path+'ohc_feats_Features')

In [50]:
# load the dataset
dataset =  Filal_SMOTE_DataSet
# retrieve the array of data
data = dataset.values
# separate into input and output columns
X = data[:, :-1].astype(str)
y = data[:, -1].astype(str)
# split the dataset into train and test sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.30, random_state=1)
print('X_train',X_train.shape,'X_test',X_test.shape)
 
print(f"Original class counts y_train: {Counter(y_train)}")
print(f"Original class counts y_test: {Counter(y_test)}")
# one-hot encode input variables
onehot_encoder = OneHotEncoder(handle_unknown='ignore')
onehot_encoder.fit(X_train)
X_train = onehot_encoder.transform(X_train)
X_test = onehot_encoder.transform(X_test)
# ordinal encode target variable
label_encoder = LabelEncoder()
label_encoder.fit(y_train)
y_train = label_encoder.transform(y_train)
y_test = label_encoder.transform(y_test)


X_train (53, 91) X_test (23, 91)
Original class counts y_train: Counter({'died=<3': 30, 'died=>3': 23})
Original class counts y_test: Counter({'died=>3': 15, 'died=<3': 8})


In [51]:
results = baseline_models(data=[X_train, X_test, y_train, y_test])
results.sort_values('f1',ascending=False)

,f1,accuracy,precision,recall
Gradient Boosting,0.651515,0.652174,0.694444,0.704167
RandomForest,0.608696,0.608696,0.670833,0.670833
DecisionTree,0.580122,0.608696,0.579365,0.583333
LogisticRegression,0.564394,0.565217,0.647321,0.637500
AdaBoostClassifier,0.564394,0.565217,0.647321,0.637500


# RandomForestClassifier

In [52]:
df_train = Filal_SMOTE_DataSet.drop('OUTCOME', axis=1)
train_labels = Filal_SMOTE_DataSet['OUTCOME']

In [53]:
df_train.shape[1]

91

In [54]:
encoders = []
features = []
for i in range(91):
    encoder_i = OneHotEncoder(handle_unknown='ignore')
    i_column = df_train.columns[i]
    encoder_i.fit(df_train[i_column].values.reshape(-1, 1))
    encoded_feature = encoder_i.transform(df_train[i_column].values.reshape(-1, 1))
    encoded_feature = encoded_feature.toarray()
    features.append(encoded_feature)
    encoders.append(encoder_i)

def get_oh(df):
    features_test = []
    for i in range(91):
        encoder_i = encoders[i]
        i_column = Filal_SMOTE_DataSet.columns[i]
        encoded_test = encoder_i.transform(df[i_column].values.reshape(-1, 1))
        encoded_test = encoded_test.toarray()
        features_test.append(encoded_test)
    return  np.concatenate(features_test, axis=1)

train_features = np.concatenate(features, axis=1)


In [55]:
train_features.shape, train_labels.values.shape

((76, 1294), (76,))

In [56]:
# separate into input and output columns
X = train_features
y = train_labels
# split the dataset into train and test sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.30, random_state=1)
# one-hot encode input variables
# ordinal encode target variable
label_encoder = LabelEncoder()
label_encoder.fit(y_train)
y_train = label_encoder.transform(y_train)
y_test = label_encoder.transform(y_test)

In [57]:
clf = RandomForestClassifier()
clf.fit(X_train, y_train)
# save the model
pickle.dump(clf, open(path+Modelname, 'wb'))